In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from src.data import data_loader, classes
from src.train import train_model
from src.model import ResNet18

In [3]:
num_classes = 10
num_epochs = 5
batch_size = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Collect baseline metrics

In [5]:
model_conv = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)

# Freeze training for all layers exept the last one for finetuning
for param in model_conv.parameters(): param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, num_classes)


model_conv = model_conv.to(device)


criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9) # Set small learning rate for finetuning
exp_lr_scheduler = None

datasets = data_loader(data_dir='data/', batch_size=batch_size)
dataset_sizes = {phase: len(datasets[phase]) for phase in ['train', 'test']}

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, device, datasets, dataset_sizes, num_epochs=num_epochs)

Epoch 0/4
----------
train Loss: 199.7974 Acc: 6.3044


KeyError: 'val'

# Custom ResNet18

In [4]:
model = ResNet18(num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model.fc.parameters(), lr=0.1, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.1)


datasets = data_loader(data_dir='data/', batch_size=batch_size)
dataset_sizes = {phase: len(datasets[phase]) for phase in ['train', 'test']}

Files already downloaded and verified
Files already downloaded and verified


In [5]:
best_custom_resnet = train_model(model, criterion, optimizer_conv, exp_lr_scheduler, device, datasets, dataset_sizes, num_epochs=num_epochs)
torch.save(best_custom_resnet.state_dict(), 'custom_resnet_best.pth')

Epoch 0/4
----------


KeyboardInterrupt: 